In [1]:
import math

FIGURES_PATH = 'out/figures/'
DATASETS_PATH = 'out/datasets/'
DICTS_PATH = 'out/dicts/'
CLUSTERS_PATH = 'out/clusters/'

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
from tqdm.notebook import tqdm
from sortedcontainers import SortedDict, SortedSet, SortedList
from multiprocesspandas import applyparallel
from pandarallel import pandarallel
import psutil
from sys import getsizeof
import networkx as nx
from scipy.cluster.hierarchy import linkage, fcluster


from netgraph import Graph, InteractiveGraph, EditableGraph

import pickle
import gc 


tqdm.pandas()
from helper import *

In [3]:
def default(mean, count, scatter):
    return (mean + abs(scatter)) / (count ** 2)

def normalize(d, target=1.0, type1=np.uint32, type2=np.float16):
   raw = sum(d.values())
   factor = target / raw
   return {(type1(key[0]), type1(key[1])): type2(value * factor) for key, value in d.items()}

def get_dists(dists, count_lower=10, dist_func=default):
    ans = dict()
    for i in dists.items():
        dist = dist_func(i[1][0], i[1][1], i[1][2])
        if (i[1][1] >= count_lower or dist != 0) and (dist >= 0):
            ans[i[0]] = dist
    return normalize(ans)

In [4]:
class Metric:
    def __init__(self, method='euclidean', max=100):
        self.method = method
        self.max = max

    def run(self, cluster1, cluster2, dists):
        self.cluster1 = cluster1
        self.cluster2 = cluster2
        self.dists = dists

        if self.method == 'euclidean':
            return self.euclidean()
        if self.method == 'min_dist':
            return self.min_dist()
        if self.method == 'max_dist':
            return self.max_dist()
        if self.method == 'average':
            return self.average()
        if self.method == 'ward':
            return self.ward()

    def _get(self, i, j):
        if i == j:
            return 0.0
        if (i, j) in self.dists:
            return self.dists[(i, j)]
        if (j, i) in self.dists:
            return self.dists[(j, i)]
        return self.max


    def euclidean(self):
        n1, n2 = len(self.cluster1), len(self.cluster2)
        s = 0.0
        for i in self.cluster1:
            for j in self.cluster2:
                s += self._get(i, j) ** 2
        return np.sqrt(s)


    def min_dist(self):
        s, mini = 0.0, self.max + 1
        for i in self.cluster1:
            for j in self.cluster2:
                s = self._get(i, j)

                if s < mini:
                    mini = s
        return mini


    def max_dist(self):
        s, maxi = 0.0, -1.0
        for i in self.cluster1:
            for j in self.cluster2:
                s = self._get(i, j)

                if s > maxi:
                    maxi = s
        return maxi


    def average(self):
        n1, n2 = len(self.cluster1), len(self.cluster2)
        s = 0.0
        for i in self.cluster1:
            for j in self.cluster2:
                s += self._get(i, j)
        return s / (n1 * n2)


    def ward(self):
        n1, n2 = len(self.cluster1), len(self.cluster2)
        s_u, s_1, s_2 = 0.0, 0.0, 0.0
        for i in self.cluster1:
            for j in self.cluster2:
                s_u += self._get(i, j) ** 2

        for i in range(n1):
            for j in range(i + 1, n1):
                s_1 += self._get(self.cluster1[i], self.cluster1[j])

        for i in range(n2):
            for j in range(i + 1, n2):
                s_2 += self._get(self.cluster2[i], self.cluster2[j])
        return (s_u - s_1 - s_2) / (n1 + n2)

In [5]:
class Elem:
    def __init__(self, i, j, val):
        self.i = i
        self.j = j
        self.val = val

    def get_key(self):
        return self.i, self.j

    def get_all(self):
        return self.i, self.j, self.val

    def __lt__(self, other):
        return self.val < other.val

    def __str__(self):
        return f"({self.i}, {self.j}): {self.val}"

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.i == other.i and self.j == other.j

class ClustersList:
    def __init__(self, fill=np.inf):
        self.dists = SortedList()
        self.fill = fill

    def insert(self, i, j, val):
        el = Elem(i, j, np.float16(val))
        self.dists.add(el)

    # def remove(self, i):
    #     for_del = []
    #     for el in self.dists:
    #         if i in el.get_key():
    #             for_del.append(el)
    #     for el in for_del:
    #         self.dists.discard(el)

    def remove(self, i, j):
        # for_del = []
        check = Elem(i, j, 0)
        for ind, el in enumerate(self.dists):
            if check == el:
                del self.dists[ind]
                # for_del.append(el)
        # for el in for_del:
        #     self.dists.discard(el)

    def __getitem__(self, i, j):
        for el in self.dists:
            i1, j1 = el.get_key()
            if i1 == i and j1 == j:
                return el.val

    def min(self):
        return self.dists.pop().get_all()


class ClustersSet:
    def __init__(self, fill=np.inf):
        self.dists = SortedSet()
        self.fill = fill

    def insert(self, i, j, val):
        el = Elem(i, j, np.float16(val))
        self.dists.add(el)

    def remove(self, i):
        for_del = []
        for el in self.dists:
            if i in el.get_key():
                for_del.append(el)
        for el in for_del:
            self.dists.discard(el)

    def min(self):
        return self.dists[0].get_key()


In [6]:
class ClustersDists:
    def __init__(self, fill=np.inf):
        self.dists = dict()
        self.fill = fill

    def set(self, i, j, val):
        self.dists[i, j] = val

    def insert(self, i, j, val):
        li = list(self.dists.items())
        is_ok = False
        for i in range(len(li)):
            if li[i][1] >= val:
                li = li[: i] + [((i, j), val)] + li[i :]
                is_ok = True
        if not is_ok:
            li.append(((i, j), val))
        self.dists = dict(li)

    def get(self, i, j):
        if (i, j) in self.dists:
            return self.dists[i, j]
        if (j, i) in self.dists:
            return self.dists[j, i]
        return self.fill

    def __getitem__(self, key):
        return self.get(key[0], key[1])

    def __setitem__(self, key, value):
        self.set(key[0], key[1], value)

    def min(self):
        return list(self.dists.items())[0][0]

    def remove(self, i):
        keys = list(self.dists.keys())
        for key in keys:
            if i in key:
                self.dists.pop(key)

In [7]:
class Clustering:
    def __init__(self, get_dists=get_dists):
        self.get_dists = get_dists
        self.statistics = {
            'min_distances': [],
            'time_of_iter': [],
            'time_of_all': 0.0,
            'count_of_iters': 0.0,
            }

    def get_stats(self):
        self.statistics['time_of_iter'] = np.array(self.statistics['time_of_iter']).mean()
        for k in self.statistics.keys():
            print(f"{k} --- {self.statistics[k]}")
        return self.statistics

    @staticmethod
    def _merge_clusters(cluster1, cluster2):
        merged_cluster = cluster1 + cluster2
        return merged_cluster

    def run(self, dists, k, top_lim):
        start0 = datetime.now()

        elements = np.unique(list(dists.keys()))[:top_lim]
        clusters = [[i] for i in elements]
        iters = len(elements) - k

        # clusters_dists = ClustersSet()
        clusters_dists = ClustersList()
        # clusters_dists = ClustersDists(fill=np.inf)

        print('Starting counting distances between clusters...')

        for i in tqdm(range(len(clusters))):
            for j in range(i + 1, len(clusters)):
                # clusters_dists.set(i, j, self.metric.run(clusters[i], clusters[j], dists))
                clusters_dists.insert(i, j, self.metric.run(clusters[i], clusters[j], dists))

        print('Starting collapsing closest clusters...')
        for _ in tqdm(range(iters)):
            start = datetime.now()

            i, j, min_distance = clusters_dists.min()
            # print(f"argmin: {datetime.now() - start}")
            # start = datetime.now()


            # min_distance = clusters_dists[(i, j)]
            merged_cluster = self._merge_clusters(clusters[i], clusters[j])

            # print(f"merge: {datetime.now() - start}")
            # start = datetime.now()


            clusters[i], clusters[j] = [], []

            clusters_dists.remove(i, j)
            # clusters_dists.remove(j)

            # print(f"remove: {datetime.now() - start}")
            # start = datetime.now()

            clusters.append(merged_cluster)

            j = len(clusters) - 1
            for i in range(len(clusters)):
                if len(clusters[i]) != 0:
                    # сюда можно паралеллизм в общий массив
                    clusters_dists.insert(i, j, self.metric.run(clusters[i], clusters[j], dists))

            # print(f"count_new: {datetime.now() - start}")
            # start = datetime.now()


            self.statistics['min_distances'].append(min_distance)
            self.statistics['time_of_iter'].append(datetime.now() - start)
        self.statistics['count_of_iters'] = iters
        self.statistics['time_of_all'] = datetime.now() - start0

        clusters = [c for c in clusters if c != []]

        return clusters

    def run_k_means(self, dists, k, max_iter=10_000):
        # Можно наканпливать minimal_dist, как внутрикластерное расстояние (в агломеративных тоже)
        # Можно сохранять среднее расстояние между кластерами и внутри кластеров, чтобы показывать на графике

        start0 = datetime.now()
        elements = np.unique(sorted(list(dists.keys()))[:25_000])

        clusters = np.random.choice(elements, k, replace=False)
        elements = elements[~np.isin(elements, clusters)]
        clusters = [[c] for c in clusters]


        print('Starting elements splitting by clusters...')
        for e in tqdm(elements):
            minimal_dist = self.metric.max * 5.0
            cluster_index = 0
            for i, c in enumerate(clusters):
                dist = self.metric.run([e], c, dists)
                if dist < minimal_dist:
                    minimal_dist = dist
                    cluster_index = i

            clusters[cluster_index].append(e)


        print('Starting operating over clusters...')
        for _ in tqdm(range(max_iter)):
            self.statistics['count_of_iters'] += 1
            start = datetime.now()
            prev_clusters = clusters.copy()
            for c1 in clusters:
                for pos_el, el in enumerate(c1):
                    minimal_dist = self.metric.max * 5.0
                    cluster_index = 0
                    for i, c in enumerate(clusters):
                        dist = self.metric.run([el], c, dists)
                        if dist < minimal_dist:
                            minimal_dist = dist
                            cluster_index = i

                    self.statistics['min_distances'].append(minimal_dist)

                    del c1[pos_el]
                    clusters[cluster_index].append(el)

            if prev_clusters == clusters:
                print('Clusters stabilizied!')
                self.statistics['time_of_all'] = datetime.now() - start0
                return clusters

            self.statistics['time_of_iter'].append(datetime.now() - start)

        print('Stopped for maximum iterations: {}'.format(max_iter))
        self.statistics['time_of_all'] = datetime.now() - start0
        return clusters


    def fit(self,
            metric: Metric,
            type='agglomerative',
            dists_path='date_distances',
            k=10,
            top_lim=10_000,
            max_iter=10_000
        ):
        with open(DATASETS_PATH + dists_path + '.pkl', 'rb') as f:
            self.dists = pickle.load(f)
        dists = self.get_dists(self.dists)
        self.metric = metric

        if type == 'agglomerative':
            return self.run(dists, k, top_lim)
        else:
            return self.run_k_means(dists, k, max_iter)

In [8]:
c = Clustering()
# clusters_euc = c.fit(metric=Metric('max_dist'), type='k_means', k=10_000, max_iter=10_000)
#
# with open(CLUSTERS_PATH + 'k_means_max_dist.pkl','wb') as f:
#      pickle.dump(clusters_euc, f)

In [9]:
clusters_euc = c.fit(metric=Metric('min_dist'), type='agglomerative', k=100, top_lim=1_000)


with open(CLUSTERS_PATH + 'ward_min_dist.pkl','wb') as f:
     pickle.dump(clusters_euc, f)


Starting counting distances between clusters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Starting collapsing closest clusters...


  0%|          | 0/900 [00:00<?, ?it/s]

argmin: 0:00:00.000016
merge: 0:00:00.000004
remove: 0:00:00.109211
count_new: 0:00:00.005853
argmin: 0:00:00.000012
merge: 0:00:00.000004
remove: 0:00:00.109223
count_new: 0:00:00.006379
argmin: 0:00:00.000009
merge: 0:00:00.000003
remove: 0:00:00.108279
count_new: 0:00:00.006845
argmin: 0:00:00.000010
merge: 0:00:00.000003
remove: 0:00:00.109132
count_new: 0:00:00.007794
argmin: 0:00:00.000011
merge: 0:00:00.000004
remove: 0:00:00.110257
count_new: 0:00:00.008580
argmin: 0:00:00.000010
merge: 0:00:00.000004
remove: 0:00:00.119355
count_new: 0:00:00.009954
argmin: 0:00:00.000010
merge: 0:00:00.000007
remove: 0:00:00.117718
count_new: 0:00:00.009010
argmin: 0:00:00.000009
merge: 0:00:00.000004
remove: 0:00:00.109881
count_new: 0:00:00.010500
argmin: 0:00:00.000010
merge: 0:00:00.000004
remove: 0:00:00.112475
count_new: 0:00:00.010540
argmin: 0:00:00.000010
merge: 0:00:00.000004
remove: 0:00:00.115137
count_new: 0:00:00.013558
argmin: 0:00:00.000010
merge: 0:00:00.000004
remove: 0:00:00

In [10]:
c.get_stats()

min_distances --- [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 1

{'min_distances': [100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  

In [11]:
# with open(CLUSTERS_PATH + 'k_means_euclidean.pkl','wb') as f:
#      pickle.dump(clusters_euc, f)